### Delete wrongly mapped recipes in planned meal table
1. get the recipes mapped to each meal plan
2. filter wrongly mapped recipes 
3. get the schedule id for worngly mapped
4. delete the wrongly mapped recipes using the API 

In [1]:
import pandas as pd
import json
import pickle
import requests
import pathlib
import time
import numpy as np
import os
import requests
from itertools import chain
import ast

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def load_json(path):
    with open(path , 'r') as f:
        return json.load(f)

def export_json(data, path='result.json'):
    with open(path, 'w') as f:
        json.dump(data, f)

def load_pickle(path):
    with open(path , 'rb') as f:
        return pickle.load(f)
    
def export_pickle(data, path='result.pikle'):
    with open(path, 'wb') as f:
        pickle.dump(data, f)

#### get wrongly mapped recipes for meal plan 

In [4]:
def get_plan_meals(plan_id):
    url = f"http://18.223.178.105:5000/api/v1/plan_management/plan/{plan_id}"
    response = requests.request("GET", url, headers={}, data="")
    return json.loads(response.text)

In [5]:
def get_meal_map_details(plan_id):
    res = get_plan_meals(plan_id)
    temp = []
    for day, vals in res['plan_details'].items():
        for tm, rep in vals.items():
            rep = rep['recipes']
            for obj in rep:
                obj['day_name'] = day
                obj['time'] = tm
                temp.append(obj)
    temp = pd.DataFrame.from_dict(temp)
    temp.recipe_name = temp.recipe_name.str.lower()
    temp = temp.groupby(['day_name', 'time'])['recipe_id'].unique()
    # temp = temp.groupby(['day_name', 'time'])['recipe_name'].unique()
    temp = temp.reset_index()
    return temp

#### get the status csv containing mapping status like pending recipes to map, wrongly mapped recipes etc  

In [6]:
def get_meal_mapping_status(df_diet_plan):
    df_result_list = []
    for plan_id, df_diet in df_diet_plan.groupby('plan_id'):
        try:
            diet_db = get_meal_map_details(plan_id)
            df_diet = df_diet.groupby(['day_name', 'time'])['recipe_id'].unique()
            df_diet = df_diet.reset_index()
            df_diet.rename(columns={'recipe_id': 'dietian_recipe_id'}, inplace=True)
            diet_db.rename(columns={'recipe_id': 'db_recipe_id'}, inplace=True)
            result= pd.merge(df_diet, diet_db, left_on=['day_name', 'time'], right_on=['day_name', 'time'], how='left')
            result.db_recipe_id = result.db_recipe_id.fillna("").apply(list)
            result['pending_recipe'] = result.apply(lambda row : list(set(row['dietian_recipe_id'])-set(row['db_recipe_id'])), axis=1)
            result['mismatch_recipe'] = result.apply(lambda row : list(set(row['db_recipe_id'])-set(row['dietian_recipe_id'])), axis=1)
            result['recipe_per_schedule'] = result.apply(lambda row : len(set(row['dietian_recipe_id'])), axis=1)
            result['mapped_recipe'] = result.apply(lambda row : len(set(row['db_recipe_id'])), axis=1)
            result['pending_recipe_per_schedule'] = result.apply(lambda row : len(set(row['pending_recipe'])), axis=1)
            result['wrong_count'] = result.apply(lambda row : len(set(row['mismatch_recipe'])), axis=1)
            result['plan_id'] = plan_id
            df_result_list.append(result)
            print('Processed ', plan_id)
        except Exception as e:
            print('\t Exception ', e)
            print('\t Failed to process ', plan_id)

    df_result = pd.concat(df_result_list, ignore_index=True)
    df_result.to_csv('meal_mapping_status.csv', index=False)
    return df_result

In [7]:
def delete_planned_meal_api(payload):
    url = "http://18.223.178.105:5000/api/v1/plan_management/delete-planned-meal"
    headers = { "Content-Type" : "application/json;charset=utf-8"}
    payload = json.dumps(payload)
    response = requests.request("POST", url, headers=headers, data=payload)
    print(response.text)

In [8]:
def delete_recipe_mapping(schedule_id, recipes):
    if recipes:
        for recipe_name in recipes:
            # payload = {"recipe_id":recipe_name_id.get(recipe_name),
            #         "schedule_id": schedule_id}
            payload = {"recipe_id":int(recipe_name),
                       "schedule_id": int(schedule_id)}
            print(payload)
            delete_planned_meal_api(payload)
            

In [9]:
def get_wrong_recipe_maps(df_diet_plan, df_meal_status):
    x = df_meal_status[['day_name', 'time', 'plan_id', 'mismatch_recipe']]
    y = df_diet_plan[['time', 'day_name', 'plan_id', 'plan_schedule_id']]
    y = y.drop_duplicates()
    
    return pd.merge(x,y, on=['time', 'day_name', 'plan_id'], how='left')

#### get wrongly mapped recipe list 

In [10]:
df_diet_plan = pd.read_csv('../../fooddb_notebooks/fooddb/csv/Meal_plan_mapping_format_28_diet_plans - All plans v5 (plan-recipe name mismatch fixed).csv')
df_diet_plan.rename(columns={'Recipe IDs':'recipe_id'}, inplace=True)
df_diet_plan = df_diet_plan[df_diet_plan['recipe_id'] != '-']
df_diet_plan.recipe_id = df_diet_plan.recipe_id.astype('int')
df_meal_status =  get_meal_mapping_status(df_diet_plan)

df_delete = get_wrong_recipe_maps(df_diet_plan, df_meal_status[df_meal_status.wrong_count > 0])

Processed  1
Processed  2
Processed  3
Processed  4
Processed  5
Processed  6
Processed  7
Processed  8
Processed  9
Processed  10
Processed  11
Processed  12
Processed  13
Processed  14
Processed  15
Processed  16
Processed  17
Processed  18
Processed  19
Processed  20
Processed  21
Processed  22
Processed  23
Processed  24
Processed  25
Processed  26
Processed  27
Processed  28


In [11]:
df_meal_status

,day_name,time,dietian_recipe_id,db_recipe_id,pending_recipe,mismatch_recipe,recipe_per_schedule,mapped_recipe,pending_recipe_per_schedule,wrong_count,plan_id
0,Friday,Breakfast,"[94, 65]","[94, 65]",[],[],2,2,0,0,1
1,Friday,Dinner,"[34, 323, 87, 21]","[34, 87, 323, 21]",[],[],4,4,0,0,1
2,Friday,Early Morning,"[274, 2]","[2, 274]",[],[],2,2,0,0,1
3,Friday,Evening Tea,"[93, 324]","[93, 324]",[],[],2,2,0,0,1
4,Friday,Lunch,"[113, 36, 97, 21]","[36, 97, 113, 21]",[],[],4,4,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
1171,Wednesday,Dinner,"[76, 226, 36]","[36, 76, 226]",[],[],3,3,0,0,28
1172,Wednesday,Early Morning,"[19, 4]","[19, 4]",[],[],2,2,0,0,28
1173,Wednesday,Evening Tea,"[267, 174, 24]","[174, 24, 267]",[],[],3,3,0,0,28
1174,Wednesday,Lunch,"[208, 226, 36, 21]","[208, 226, 36, 21]",[],[],4,4,0,0,28


#### delete wrongly mapped recipes 

In [ ]:
df_delete.apply(lambda row: delete_recipe_mapping(row['plan_schedule_id'], row['mismatch_recipe']), axis=1)